# TP 2 Multitâches
# Exercice 1 - Pi Monte Carlo

## Introduction

Cet exercice nécessite le package **Numba**. 

Ce TP a pour but de paralléliser l'algorithme de pi par Monte Carlo en utilisant **multithreading** et **multiprocessing**.

La méthode `pick` tire `n` coups dans le carré $[-1,1] \times [-1,1]$ et retourne le nombre de coups tirés dans le disque inscrit au carré.

In [132]:
import numpy as np
from numba import njit
from concurrent.futures import ProcessPoolExecutor
from concurrent.futures import ThreadPoolExecutor
import time


In [76]:
@njit
def pick(n):
    '''
    input : n nombre de tirage dans le carre [-1, 1]
    output: count_inside nombre de coups tires dans le disque inscrit au carre
    '''
    count_inside = 0
    for i in range(n):
        x, y = np.random.random(2) * 2 - 1
        if x**2 + y**2 <= 1: count_inside += 1
    return count_inside

La méthode pi_mc appel la méthode pick sur la valeur n et retourne la valeur approchée $\pi$ par la formule $4 \times p_C/p_T$ où $p_C$ désigne le nombre de coups dans le disque et $p_T$ le nombre de coups total.

In [24]:
def pi_mc(n):
    '''
    input : n nombre de tirage dans le carré [-1, 1]
    output : api : valeur de pi calculée par Monte Carlo
    '''
    api = 4 * pick(n) / n
    return api

In [136]:
tic = time.time()
res=pi_mc(10000000)
toc = time.time()
print(f'Pour 10000 tir, l approximation de pi est {res} et le temps est  {toc - tic} seconds')

Pour 10000 tir, l approximation de pi est 3.1415888 et le temps est  1.3483788967132568 seconds



## 1 - Parallélisation avec multiprocessing

1.a) Sur la base de `pi_mc` créer une fonction `pi_mc_mp` qui répartit le travail entre plusieurs processus à l'aide de `multiprocessing` comme vu en cours.


In [137]:
def pi_mc_mp(n,process):
    p_exe = ProcessPoolExecutor(process)
    tic = time.time()
    tir=[n//10 for i in range(10)]
    result = [res for res in p_exe.map(pick,tir)]
    count = sum(result)
    pi = 4 * count / n
    toc = time.time()
    print(f'Pour {n} tir et {process} processus, l approximation de pi est {pi} et le temps est  {toc - tic} seconds')

In [139]:
pi_mc_mp(10000000,2)

Pour 10000000 tir et 2 processus, l approximation de pi est 3.1421304 et le temps est  0.7069547176361084 seconds


In [140]:
pi_mc_mp(10000000,4)

Pour 10000000 tir et 4 processus, l approximation de pi est 3.1416536 et le temps est  0.4667181968688965 seconds


In [141]:
pi_mc_mp(10000000,6)

Pour 10000000 tir et 6 processus, l approximation de pi est 3.141664 et le temps est  0.3641955852508545 seconds


1.b) Mesurer les temps de restitution en variant le nombre de tir et le nombre de processus.


## 2 - Parallélisation avec multithreading`

2.a) Sur la base de `pi_mc_mp` créer une fonction `pi_mc_mt` qui répartit le travail entre plusieurs threads à l'aide de `multithreading` comme vu en cours.


In [186]:
nogil=True
njit()
def pick(n):
    '''
    input : n nombre de tirage dans le carre [-1, 1]
    output: count_inside nombre de coups tires dans le disque inscrit au carre
    '''
    count_inside = 0
    for i in range(n):
        x, y = np.random.random(2) * 2 - 1
        if x**2 + y**2 <= 1: count_inside += 1
    return count_inside

In [187]:
def pi_mc_mt(n,work):
    p_exe = ThreadPoolExecutor(work)
    tic = time.time()
    tir=[n//100 for i in range(100)]
    result = [res for res in p_exe.map(pick,tir)]
    count = sum(result)
    pi = 4 * count / n
    toc = time.time()
    print(f'Pour {n} tir, l approximation de pi est {pi} et le temps est  {toc - tic} seconds')

2.b) Mesurer les temps de restitution en variant le nombre de tir et le nombre de processus. Comparer avec la méthode précédente.

In [201]:
pi_mc_mt(100000,1)

Pour 100000 tir, l approximation de pi est 3.14672 et le temps est  0.37597131729125977 seconds


In [202]:
pi_mc_mt(10000,2)

Pour 10000 tir, l approximation de pi est 3.1176 et le temps est  0.1213982105255127 seconds


In [203]:
pi_mc_mt(10000,4)

Pour 10000 tir, l approximation de pi est 3.1636 et le temps est  0.1097404956817627 seconds


In [204]:
pi_mc_mt(10000,3)

Pour 10000 tir, l approximation de pi est 3.1608 et le temps est  0.1117088794708252 seconds


## J'ai remarqué que le multuprocessing est plus rapide que le multithreading pour ce probleme